In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numba import njit
from scipy import sparse

from graph_ml.datasets.polbooks import PolBooks
from graph_ml.utils import utils
# graph = nx.erdos_renyi_graph(10, 0.2, seed=92)
# A = nx.adjacency_matrix(graph)
# nx.draw(graph)
# np.array(A.todense())

In [2]:
A, group_membership = PolBooks().adj, PolBooks().y
A.toarray(), group_membership

Using existing file polbooks.zip
Extracting /tmp/polbooks.zip
Using existing file polbooks.zip
Extracting /tmp/polbooks.zip


(array([[0, 1, 1, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 1, 0]]),
 array([0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0]))

In [3]:
window_length = 10
dcsbm = True
group_membership = np.unique(group_membership, return_inverse=True)[1]
group_membership

array([0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0])

In [4]:
n_nodes = A.shape[0]
node2group = utils.to_member_matrix(group_membership)

In [5]:
indeg = np.array(A.sum(axis=1)).reshape(-1)
indeg

array([ 6,  4,  4, 23,  8,  7, 11,  8, 25, 16, 15, 18, 25, 13,  9,  5,  3,
        5,  3,  5, 10,  5,  7,  9,  9,  5,  9,  9,  3,  4, 20, 11,  5,  9,
        5, 10,  5,  7,  7,  8, 18,  8,  6,  5,  5,  6,  4, 18,  4,  8,  3,
        6,  5,  5,  6,  4,  4,  5, 13,  5,  6,  4,  6,  4,  9,  7, 21,  6,
        4,  5,  7, 15, 22, 21, 16, 16, 13,  7,  5,  8,  4,  4,  9,  8, 23,
        5, 14,  5,  5,  6,  5,  8,  3,  7,  7,  4,  6,  6,  5, 12, 12,  5,
        4,  2,  3])

In [6]:

Lambda = (node2group.T @ A @ node2group).toarray()
Din = np.array(np.sum(Lambda, axis=0)).reshape(-1)
Nin = np.array(node2group.sum(axis=0)).reshape(-1)
Psbm = np.einsum('ij,i->ij', Lambda, 1 / np.maximum(1, np.array(np.sum(Lambda, axis=1))))
Psbm_pow = utils.matrix_sum_power(Psbm, window_length) / window_length

block2node = (
    sparse.diags(1 / np.maximum(1, Din))
    @ sparse.csr_matrix(
        node2group.T
    )
    @ sparse.diags(indeg)
)




In [7]:
block2block = sparse.csr_matrix(
    Psbm_pow
)
block2block.data = utils.csr_row_cumsum(block2block.indptr, block2block.data)

block2node.data = utils.csr_row_cumsum(block2node.indptr, block2node.data)

In [8]:
block2node.toarray().shape

(3, 105)

In [9]:
block2block.toarray()

array([[0.10477346, 0.60810079, 1.        ],
       [0.08979549, 0.8052493 , 1.        ],
       [0.07837984, 0.29670563, 1.        ]])

In [10]:
from graph_ml.transformations import samplers

In [11]:
sampler = samplers.SbmNodeSampler(group_membership=group_membership, dcsbm=dcsbm)
sampler.fit(A)

type A  <class 'scipy.sparse._csr.csr_array'>   (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (1, 0)	1
  (1, 3)	1
  (1, 5)	1
  (1, 6)	1
  (2, 0)	1
  (2, 4)	1
  (2, 5)	1
  (2, 7)	1
  (3, 0)	1
  (3, 1)	1
  (3, 5)	1
  (3, 8)	1
  (3, 9)	1
  (3, 10)	1
  (3, 11)	1
  (3, 12)	1
  (3, 13)	1
  (3, 14)	1
  (3, 15)	1
  :	:
  (100, 66)	1
  (100, 73)	1
  (100, 79)	1
  (100, 83)	1
  (100, 84)	1
  (100, 86)	1
  (100, 91)	1
  (100, 96)	1
  (100, 98)	1
  (100, 99)	1
  (100, 101)	1
  (101, 61)	1
  (101, 84)	1
  (101, 86)	1
  (101, 94)	1
  (101, 100)	1
  (102, 46)	1
  (102, 93)	1
  (102, 94)	1
  (102, 95)	1
  (103, 67)	1
  (103, 104)	1
  (104, 67)	1
  (104, 69)	1
  (104, 103)	1


In [12]:
sampler.block2block.shape

(3, 3)

In [13]:
sampler.block2node.shape

(3, 105)

In [14]:
sampler.node2group.shape

(105, 3)

In [15]:
centers = np.arange(105)
sampler.sample(centers, 10, 10)

array([ 38,  47,  12,  47,  12,  99,  25,  15,  35,  11,  58,  77,  47,
        11,  17,  96,  81,  37,   3,  40,  12,  42,  29,  23,  32,  29,
        37,  40,   1,   3,  84,  30,  12,  35,  62,  35,  84,   9,  53,
        19,  11,  55,   3,  72,  15,  13,  31,   7,  84,   5,  74,  11,
         1,  82,   3,  24,  43,  88,  40,  81,  36,  86,  72,  82,  75,
        47,  94,  99,  86,  47,  20,  70,  66,  87,  71,  30,   8,   9,
        95,  70,  20,  75,  31,  72,  27,  75,  84,  68,  12,  64, 100,
        81,   7,  82,  97,  62,  70,  94,  86, 102,  60,  94, 101,   8,
        48])

In [18]:
sampler.sample(centers, 10, 10).shape

(105,)

In [19]:
centers

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104])